# Exploratory Data Analysis (EDA) – FDA Drug Event API

This notebook is for exploring the structure and content of data returned from the openFDA Drug Event API.

Goals:
- Understand the response format
- Identify key fields of interest (e.g., drug names, patient demographics, outcomes)
- Evaluate what preprocessing or flattening is needed
- Determine what dimensions and facts we might extract for the pipeline


In [26]:
import requests
import pandas as pd
import json

In [27]:

# Define base URL and parameters
url = "https://api.fda.gov/drug/event.json"
params = {
    "limit": 100  # just the first page for now
     #"sort": "receivedate:desc"   # order by date descending
}

# Send request
response = requests.get(url, params=params)
data = response.json()

print("data type is: " + str(type(data)))  # should be a dict

# Check top-level structure
print("top-level structure: " + str(data.keys())) 

# should show ['meta', 'results']
print("The number of records in the call: " + str(len(data["results"]))) # number of records

# Pretty print the first record
print(json.dumps(data["results"][0], indent=2))

#print(data["results"][0])


data type is: <class 'dict'>
top-level structure: dict_keys(['meta', 'results'])
The number of records in the call: 100
{
  "safetyreportid": "5801206-7",
  "transmissiondateformat": "102",
  "transmissiondate": "20090109",
  "serious": "1",
  "seriousnessdeath": "1",
  "receivedateformat": "102",
  "receivedate": "20080707",
  "receiptdateformat": "102",
  "receiptdate": "20080625",
  "fulfillexpeditecriteria": "1",
  "companynumb": "JACAN16471",
  "primarysource": {
    "reportercountry": "CANADA",
    "qualification": "3"
  },
  "sender": {
    "senderorganization": "FDA-Public Use"
  },
  "receiver": null,
  "patient": {
    "patientonsetage": "26",
    "patientonsetageunit": "801",
    "patientsex": "1",
    "patientdeath": {
      "patientdeathdateformat": null,
      "patientdeathdate": null
    },
    "reaction": [
      {
        "reactionmeddrapt": "DRUG ADMINISTRATION ERROR"
      },
      {
        "reactionmeddrapt": "OVERDOSE"
      }
    ],
    "drug": [
      {
        

In [28]:
df_reports_messy = pd.json_normalize(data["results"])
df_reports_messy.head()

df_reports_messy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 39 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   safetyreportid                               100 non-null    object 
 1   transmissiondateformat                       100 non-null    object 
 2   transmissiondate                             100 non-null    object 
 3   serious                                      100 non-null    object 
 4   seriousnessdeath                             4 non-null      object 
 5   receivedateformat                            100 non-null    object 
 6   receivedate                                  100 non-null    object 
 7   receiptdateformat                            100 non-null    object 
 8   receiptdate                                  100 non-null    object 
 9   fulfillexpeditecriteria                      100 non-null    object 
 10  com

In [29]:
df_reports_messy["report_id"] = df_reports_messy.index


In [30]:
df_reports_messy.head()

,safetyreportid,transmissiondateformat,transmissiondate,serious,seriousnessdeath,receivedateformat,receivedate,receiptdateformat,receiptdate,fulfillexpeditecriteria,...,receiver.receivertype,receiver.receiverorganization,seriousnessother,occurcountry,patient.patientagegroup,seriousnesshospitalization,patient.summary.narrativeincludeclinical,seriousnesslifethreatening,patient.patientweight,report_id
0,5801206-7,102,20090109,1,1,102,20080707,102,20080625,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,10003300,102,20141002,1,NaN,102,20140306,102,20140306,2,...,6,FDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,10003301,102,20141002,1,NaN,102,20140228,102,20140228,2,...,6,FDA,1,NaN,NaN,NaN,NaN,NaN,NaN,2
3,10003302,102,20141002,2,NaN,102,20140312,102,20140312,2,...,6,FDA,NaN,US,NaN,NaN,NaN,NaN,NaN,3
4,10003304,102,20141212,2,NaN,102,20140312,102,20140424,2,...,6,FDA,NaN,US,NaN,NaN,NaN,NaN,NaN,4


In [31]:
number_of_reports = len(df_reports_messy["report_id"].unique())
print(number_of_reports)

number_of_safety_reports = len(df_reports_messy["safetyreportid"].unique())
number_of_safety_reports

100


100

In [32]:
df_reports_messy.columns.tolist()

['safetyreportid',
 'transmissiondateformat',
 'transmissiondate',
 'serious',
 'seriousnessdeath',
 'receivedateformat',
 'receivedate',
 'receiptdateformat',
 'receiptdate',
 'fulfillexpeditecriteria',
 'companynumb',
 'receiver',
 'primarysource.reportercountry',
 'primarysource.qualification',
 'sender.senderorganization',
 'patient.patientonsetage',
 'patient.patientonsetageunit',
 'patient.patientsex',
 'patient.patientdeath.patientdeathdateformat',
 'patient.patientdeath.patientdeathdate',
 'patient.reaction',
 'patient.drug',
 'safetyreportversion',
 'primarysourcecountry',
 'reporttype',
 'seriousnessdisabling',
 'duplicate',
 'reportduplicate.duplicatesource',
 'reportduplicate.duplicatenumb',
 'sender.sendertype',
 'receiver.receivertype',
 'receiver.receiverorganization',
 'seriousnessother',
 'occurcountry',
 'patient.patientagegroup',
 'seriousnesshospitalization',
 'patient.summary.narrativeincludeclinical',
 'seriousnesslifethreatening',
 'patient.patientweight',
 'repo

In [33]:
report_columns_to_keep = [
    "report_id",
    "receivedate",
    "seriousnessdeath",
    "seriousnesshospitalization",
    "seriousnesslifethreatening",
    "seriousnessdisabling",
    "seriousnessother",
    "patient.patientonsetage",
    "patient.patientonsetageunit",
    "patient.patientsex",
    'patient.patientweight'
]

df_reports = df_reports_messy[report_columns_to_keep]
df_reports.head()

,report_id,receivedate,seriousnessdeath,seriousnesshospitalization,seriousnesslifethreatening,seriousnessdisabling,seriousnessother,patient.patientonsetage,patient.patientonsetageunit,patient.patientsex,patient.patientweight
0,0,20080707,1,NaN,NaN,NaN,NaN,26,801,1,NaN
1,1,20140306,NaN,NaN,NaN,1,NaN,77,801,2,NaN
2,2,20140228,NaN,NaN,NaN,NaN,1,NaN,NaN,2,NaN
3,3,20140312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
4,4,20140312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN


In [34]:
df_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   report_id                    100 non-null    int64 
 1   receivedate                  100 non-null    object
 2   seriousnessdeath             4 non-null      object
 3   seriousnesshospitalization   21 non-null     object
 4   seriousnesslifethreatening   3 non-null      object
 5   seriousnessdisabling         2 non-null      object
 6   seriousnessother             14 non-null     object
 7   patient.patientonsetage      89 non-null     object
 8   patient.patientonsetageunit  89 non-null     object
 9   patient.patientsex           100 non-null    object
 10  patient.patientweight        11 non-null     object
dtypes: int64(1), object(10)
memory usage: 8.7+ KB


In [35]:
df_reports["receivedate"] = pd.to_datetime(df_reports["receivedate"], errors="coerce")
df_reports["receivedate"].head()

C:\Users\aqora\AppData\Local\Temp\ipykernel_18656\1124421778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reports["receivedate"] = pd.to_datetime(df_reports["receivedate"], errors="coerce")


0   2008-07-07
1   2014-03-06
2   2014-02-28
3   2014-03-12
4   2014-03-12
Name: receivedate, dtype: datetime64[ns]

In [36]:
serious_cols = [
    "seriousnessdeath",
    "seriousnesshospitalization",
    "seriousnesslifethreatening",
    "seriousnessdisabling",
    "seriousnessother"
]

for cols in serious_cols:
    df_reports.loc[:,cols] = df_reports[cols].fillna(0).astype(int)
    
df_reports.head()

,report_id,receivedate,seriousnessdeath,seriousnesshospitalization,seriousnesslifethreatening,seriousnessdisabling,seriousnessother,patient.patientonsetage,patient.patientonsetageunit,patient.patientsex,patient.patientweight
0,0,2008-07-07,1,0,0,0,0,26,801,1,NaN
1,1,2014-03-06,0,0,0,1,0,77,801,2,NaN
2,2,2014-02-28,0,0,0,0,1,NaN,NaN,2,NaN
3,3,2014-03-12,0,0,0,0,0,NaN,NaN,1,NaN
4,4,2014-03-12,0,0,0,0,0,NaN,NaN,2,NaN


In [37]:
sex_mapping = { "1":"male", "2":"female"}
df_reports.loc[:,"patient.patientsex"] = df_reports["patient.patientsex"].map(sex_mapping).fillna("unknown")

In [38]:
# show me the patient weights

patient_weights = df_reports[df_reports["patient.patientweight"].notnull()]
patient_weights = patient_weights[["report_id", "patient.patientweight"]]
patient_weights

,report_id,patient.patientweight
12,12,41.8
17,17,65.5
18,18,57.8
23,23,57
29,29,61
34,34,66
36,36,97.96
51,51,120
57,57,28
75,75,88


In [39]:
df_reports["patient.patientweight"] = pd.to_numeric(df_reports["patient.patientweight"], errors="coerce")

df_reports["patient.patientonsetage"] = pd.to_numeric(df_reports["patient.patientonsetage"], errors="coerce")

#df_reports_filtered.loc[:,"patient.patientonsetage"] = pd.to_numeric(df_reports_filtered["patient.patientonsetage"], errors="coerce")


#df_reports_filtered.info()

C:\Users\aqora\AppData\Local\Temp\ipykernel_18656\926173370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reports["patient.patientweight"] = pd.to_numeric(df_reports["patient.patientweight"], errors="coerce")
C:\Users\aqora\AppData\Local\Temp\ipykernel_18656\926173370.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reports["patient.patientonsetage"] = pd.to_numeric(df_reports["patient.patientonsetage"], errors="coerce")


In [40]:
df_reports.rename(columns={
    "patient.patientonsetage": "patient_onset_age",
    "patient.patientonsetageunit": "patient_onset_age_unit",
    "patient.patientsex": "patient_sex",
    "patient.patientweight": "patient_weight"
}, inplace=True
)
df_reports.head()

C:\Users\aqora\AppData\Local\Temp\ipykernel_18656\1247883352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reports.rename(columns={


,report_id,receivedate,seriousnessdeath,seriousnesshospitalization,seriousnesslifethreatening,seriousnessdisabling,seriousnessother,patient_onset_age,patient_onset_age_unit,patient_sex,patient_weight
0,0,2008-07-07,1,0,0,0,0,26.0,801,male,NaN
1,1,2014-03-06,0,0,0,1,0,77.0,801,female,NaN
2,2,2014-02-28,0,0,0,0,1,NaN,NaN,female,NaN
3,3,2014-03-12,0,0,0,0,0,NaN,NaN,male,NaN
4,4,2014-03-12,0,0,0,0,0,NaN,NaN,female,NaN


In [41]:
df_reports_messy.head(1)

,safetyreportid,transmissiondateformat,transmissiondate,serious,seriousnessdeath,receivedateformat,receivedate,receiptdateformat,receiptdate,fulfillexpeditecriteria,...,receiver.receivertype,receiver.receiverorganization,seriousnessother,occurcountry,patient.patientagegroup,seriousnesshospitalization,patient.summary.narrativeincludeclinical,seriousnesslifethreatening,patient.patientweight,report_id
0,5801206-7,102,20090109,1,1,102,20080707,102,20080625,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [46]:
df_drugs =  df_reports_messy[["report_id","patient.drug"]]#.explode().reset_index(drop=True)
df_drugs.head()


,report_id,patient.drug
0,0,"[{'drugcharacterization': '1', 'medicinalprodu..."
1,1,"[{'drugcharacterization': '1', 'medicinalprodu..."
2,2,"[{'drugcharacterization': '1', 'medicinalprodu..."
3,3,"[{'drugcharacterization': '1', 'medicinalprodu..."
4,4,"[{'drugcharacterization': '1', 'medicinalprodu..."


In [47]:
df_drugs_exploded = df_drugs.explode("patient.drug").reset_index(drop=True)
df_drugs_exploded.head()

,report_id,patient.drug
0,0,"{'drugcharacterization': '1', 'medicinalproduc..."
1,1,"{'drugcharacterization': '1', 'medicinalproduc..."
2,2,"{'drugcharacterization': '1', 'medicinalproduc..."
3,3,"{'drugcharacterization': '1', 'medicinalproduc..."
4,4,"{'drugcharacterization': '1', 'medicinalproduc..."


In [48]:
drug_details = pd.json_normalize(df_drugs_exploded["patient.drug"])
drug_details.head()

,drugcharacterization,medicinalproduct,drugauthorizationnumb,drugadministrationroute,drugindication,drugbatchnumb,drugstructuredosagenumb,drugstructuredosageunit,drugdosagetext,drugstartdateformat,...,drugdosageform,openfda.pharm_class_pe,activesubstance.activesubstancename,drugenddateformat,drugenddate,drugseparatedosagenumb,drugintervaldosageunitnumb,drugintervaldosagedefinition,drugcumulativedosagenumb,drugcumulativedosageunit
0,1,DURAGESIC-100,019813,041,DRUG ABUSE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,BONIVA,021858,042,OSTEOPOROSIS,H6200HO3,3,003,"3 MG, 1 IN 3 M, INTRAVENOUS (NOT OTHERWISE SPE...",102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,IBUPROFEN,17463,NaN,PRODUCT USED FOR UNKNOWN INDICATION,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,LYRICA,021446,NaN,NaN,NaN,NaN,NaN,UNK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,DOXYCYCLINE HYCLATE,050007,NaN,NaN,NaN,NaN,NaN,UNK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_drugs = pd.concat([df_drugs_exploded["report_id"], drug_details], axis=1)

Number of unique reports in df_drugs: 100
